# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="examples/line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="examples/laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, consult the forums for more troubleshooting tips.**  

## Import Packages

In [1]:
#importing some useful packages
import numpy as np
import cv2
import os#to access files and directories and to create them
import shutil#to remove old output directory and contents

## Read in an Image

In [2]:
#reading in an image
absoluteImgPath=os.path.join(os.getcwd(),'test_images','solidWhiteRight.jpg')
testReadImage = cv2.imread(absoluteImgPath)

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [3]:
def detectLines(img):
    '''This function accepts an image containing lane lines and returns that image with the lane lines marked'''
    #convert to greyscale
    grayImg=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    #apply gaussian blur(only odd kernelSizes)
    grayBlur=cv2.GaussianBlur(grayImg,(5,5),0)
    
    #run canny algorithm on greyscale img(it also does gaussian blur)
    edges=cv2.Canny(grayBlur,50,125)
    
    #create image with only masked section of edges to run hough on
    mask=np.zeros_like(edges)
    
    #fillpoly makes maskimg all white(3rd argument) in are definded with vertices(2nd argument)
    cv2.fillPoly(mask,np.array([[(0,img.shape[0]),(12*img.shape[1]/25,3*img.shape[0]/5),(52.5*img.shape[1]/100,3*img.shape[0]/5),(img.shape[1],img.shape[0])]],dtype=np.int32),255)
    maskedEdges=cv2.bitwise_and(edges,mask)
    
    #get lines using houghTransformation on maskedEdges img cv2.HoughLinesP(image,rhoRes,angleRes,threshold,np.array([]),min_line_length,max_line_gap)
    lines=cv2.HoughLinesP(maskedEdges,2,np.pi/180,15,np.array([]),30,20)

    #create image to mark lines on
    lineImg=np.copy(img)

    #get 2 points for each line to draw
    #these will store points from lines closest to camera and furthest from camera for each lane line(initilize them to points that will be overwritten)
    rLEnd=[-1,img.shape[0]+1]#off bottom of screen
    rLStart=[-1,-1]#off top of screen
    lLEnd=[-1,img.shape[0]+1]#off bottom of screen
    lLStart=[-1,-1]#off top of screen
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            #check if points should be a new endpoint for right or left lines and which end
            slope=0
            if(x2 != x1):#would get divide by 0 error before adding this
                slope=(y2-y1)/(x2-x1)
            if (slope<-.2)and(x1<img.shape[1]/2):#left line segments
                if y1<y2:
                    if y1<lLEnd[1]:
                        lLEnd=[x1,y1]
                    if y2>lLStart[1]:
                        lLStart=[x2,y2]
                else: #y2<y1 or y2==y1 (shouldn't happen since line's slope is not 0)
                    if y2<lLEnd[1]:
                        lLEnd=[x2,y2]
                    if y1>lLStart[1]:
                        lLStart=[x1,y1]
            if (slope>.2)and(x1>img.shape[1]/2):#right line segments
                if y1<y2:
                    if y1<rLEnd[1]:
                        rLEnd=[x1,y1]
                    if y2>rLStart[1]:
                        rLStart=[x2,y2]
                else: #y2<y1 or y2==y1(which shouldn't happen since line's slope is not 0)
                    if y2<rLEnd[1]:
                        rLEnd=[x2,y2]
                    if y1>rLStart[1]:
                        rLStart=[x1,y1]

    #calculate slope and intercept for lines(using endpoints) then draw lines using points y=top of region and y=bottom of region
    leftSlope=(lLEnd[1]-lLStart[1])/(lLEnd[0]-lLStart[0])
    rightSlope=(rLEnd[1]-rLStart[1])/(rLEnd[0]-rLStart[0])
    leftInter=lLStart[1]-(lLStart[0]*leftSlope)
    rightInter=rLStart[1]-(rLStart[0]*rightSlope)
    #y=mx+b so x=(y-b)/m
    
    #middle line(for testing purposes and make it clearer what is going on)
    cv2.line(lineImg,(int(img.shape[1]/2),img.shape[0]),(int(img.shape[1]/2),0),(0,0,255),2)
    
    #extrapolate the line at y=img.shape[0] and y=3*img.shape[0]/5
    cv2.line(lineImg,(int((img.shape[0]-leftInter)/leftSlope),img.shape[0]),(int((3*img.shape[0]/5-leftInter)/leftSlope),int(3*img.shape[0]/5)),(0,255,0),2)#left line
    cv2.line(lineImg,(int((img.shape[0]-rightInter)/rightSlope),img.shape[0]),(int((3*img.shape[0]/5-rightInter)/rightSlope),int(3*img.shape[0]/5)),(255,0,0),2)#right line
    return lineImg

## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [4]:
def imgDir(dir):
    '''dir is path to directory where all the images(and only the images) are stored
    images can be any format, and results are outputed same type as input
    '''
    #load filenames of all the images want to detectLines in and create output folder
    filenames=os.listdir(dir)
    if os.path.exists(dir+'_output'):
        shutil.rmtree(dir+'_output')
    os.mkdir(os.path.join(os.getcwd(),dir+'_output'))
    
    for img in filenames:
        absImgPath=os.path.join(os.getcwd(),dir,img)
        outImg=detectLines(cv2.imread(absImgPath))#read in the image and detect lane lines
        cv2.imwrite(dir+'_output/'+img,outImg)#save image with lane lines to output directory
        cv2.imshow(img,outImg)#show image with lane lines to screen
        while True:
            if cv2.waitKey(1) & 0xFF==ord('q'):#when they hit 'q' close the image on screen and move to next one
                break
        cv2.destroyAllWindows()

In [5]:
imgDir('test_images')#test the function imgDir on the test_images directory

# Test on Videos
You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

solidWhiteRight.mp4

solidYellowLeft.mp4

In [6]:
def videoDir(dir):
    '''dir is path to directory containing multiple videos to process,
    vid can be any format(that your computer has the proper codecs for)
    this will output whatever video type vid is'''
    #load filenames of all the videos want to detectLines in and create output folder
    filenames=os.listdir(dir)
    if os.path.exists(dir+'_output'):
        shutil.rmtree(dir+'_output')
    os.mkdir(os.path.join(os.getcwd(),dir+'_output'))
    
    for vid in filenames:
        absVidPath=os.path.join(os.getcwd(),dir,vid)
        cap=cv2.VideoCapture(absVidPath)
        
        #create VideoWriter object to create video file with same metadata as vid and put it into the output directory
        absOutVidPath=os.path.join(os.getcwd(),dir+'_output',vid)
        out=cv2.VideoWriter(absOutVidPath,int(cap.get(cv2.CAP_PROP_FOURCC)),cap.get(cv2.CAP_PROP_FPS),(int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))
        
        while(cap.isOpened()):#while vid still playing
            ret,frame=cap.read()
            if ret==True:
                outFrame=detectLines(frame)#detect lane lines in the frame
                out.write(outFrame)#write frame with lane lines drawn on to output video
                cv2.imshow(vid,outFrame)#show fram with lane lines to screen
                if cv2.waitKey(1) & 0xFF==ord('q'):#stop processing this video when user hits 'q'
                    break
            else:
                break
        #once done close streams
        cap.release()
        out.release()
        #close window playing video
        cv2.destroyAllWindows()

In [7]:
videoDir('test_videos')#test videoDir on the test_videos directory

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

When it goes over light part of the road the lines go crazy

In [8]:
dir='test_videos'
vid='challenge.mp4'
absVidPath=os.path.join(os.getcwd(),dir,vid)
cap=cv2.VideoCapture(absVidPath)

#create VideoWriter object to create video file with same metadata as vid and put it into the output directory
absOutVidPath=os.path.join(os.getcwd(),dir+'_output',vid)
out=cv2.VideoWriter(absOutVidPath,int(cap.get(cv2.CAP_PROP_FOURCC)),cap.get(cv2.CAP_PROP_FPS),(int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))
while(cap.isOpened()):#while vid still playing
    ret,frame=cap.read()
    if ret==True:
        outFrame=detectLines(frame)#detect lane lines in the frame
        out.write(outFrame)#write frame with lane lines drawn on to output video
        cv2.imshow(vid,outFrame)#show fram with lane lines to screen
        if cv2.waitKey(1) & 0xFF==ord('q'):#stop processing this video when user hits 'q'
            break
    else:
        break
#once done close streams
cap.release()
out.release()
#close window playing video
cv2.destroyAllWindows()